# ReDimNetNoMel Disable bad layers

In [1]:
import torch
print(torch.__version__)
from torchsummary import summary


import torchaudio
import torchaudio.transforms as T

import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from math import floor
import copy


2.6.0+cu124


In [2]:
model_name='B0'
# train_type='ft_lm'
train_type='ptn'
dataset='vox2'

torch.hub.set_dir('/data/deep/redimnet/models')

original_model = torch.hub.load('IDRnD/ReDimNet', 'ReDimNet', 
                       model_name=model_name, 
                       train_type=train_type, 
                       dataset=dataset)

Using cache found in /data/deep/redimnet/models/IDRnD_ReDimNet_master


/data/deep/redimnet/models/IDRnD_ReDimNet_master
load_res : <All keys matched successfully>


*  ReDimNetWrap expects raw 16 kHz mono audio, exactly 32 000 samples

In [3]:
from torchinfo import summary
summary(original_model, input_size=(1, 32000))

/data/deep/redimnet/models/IDRnD_ReDimNet_master/redimnet/layers/features.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetWrap                                                 [1, 192]                  --
├─MelBanks: 1-1                                              [1, 60, 134]              --
│    └─Sequential: 2-1                                       [1, 60, 134]              --
│    │    └─Identity: 3-1                                    [1, 32000]                --
│    │    └─PreEmphasis: 3-2                                 [1, 32000]                --
│    │    └─MelSpectrogram: 3-3                              [1, 60, 134]              --
├─ReDimNet: 1-2                                              [1, 600, 134]             --
│    └─Sequential: 2-2                                       [1, 600, 134]             --
│    │    └─Conv2d: 3-4                                      [1, 10, 60, 134]          100
│    │    └─LayerNorm: 3-5                                   [1, 10, 60, 134]          20
│   

we can see MelSpectrogram inside the model ; lets take it outside the model;


In [4]:
for name, module in original_model.named_children():
    print(name, "=>", module)

backbone => ReDimNet(
  (stem): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
    (2): to1d()
  )
  (stage0): Sequential(
    (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
    (1): to2d(f=60,c=10)
    (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
    (3): ConvBlock2d(
      (conv_block): ResBasicBlock(
        (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      

# create new model

In [5]:

class Conv1dAs2d(nn.Module):
    """
    Replace a Conv1d with an equivalent Conv2d (H = kernel, W = 1)
    so that ONNX shows only Conv2d, which RKNN supports.
    """
    def __init__(self, conv1d: nn.Conv1d):
        super().__init__()

        k, d, s, g = conv1d.kernel_size[0], conv1d.dilation[0], conv1d.stride[0], conv1d.groups

        # --- numeric padding ---
        if isinstance(conv1d.padding, str):        # "same" or "valid"
            if conv1d.padding == "same":
                pad_num = floor(d * (k - 1) / 2)
            else:                                  # "valid"
                pad_num = 0
        else:                                      # already a tuple/int
            pad_num = conv1d.padding[0]

        # Build the Conv2d with weights copied
        self.conv2d = nn.Conv2d(
            in_channels  = conv1d.in_channels,
            out_channels = conv1d.out_channels,
            kernel_size  = (k, 1),
            stride       = (s, 1),
            padding      = (pad_num, 0),
            dilation     = (d, 1),
            groups       = g,
            bias         = conv1d.bias is not None
        )

        with torch.no_grad():
            # (out, in/groups, k) → (out, in/groups, k, 1)
            self.conv2d.weight.copy_(conv1d.weight.unsqueeze(-1))
            if conv1d.bias is not None:
                self.conv2d.bias.copy_(conv1d.bias)

    def forward(self, x):           # x: [B, C, T]
        #todo : pay attention to the input shape! AVI APPROVED
        return self.conv2d(x.unsqueeze(-1)).squeeze(-1)


In [6]:
########################################
# 2) Define a Model Class without MelBanks
########################################
import torch
import torch.nn as nn

class ReDimNetNoMel(nn.Module):
    """
    A wrapper around the original ReDimNetWrap that:
      - Excludes the 'spec' (MelBanks) module
      - Uses 'backbone', 'pool', 'bn', and 'linear'
    We expect a precomputed mel spectrogram as input with shape [B, 1, n_mels, time_frames].
    """
    def __init__(self, original_wrap):
        super().__init__()
        
        # Grab references to the submodules we want to keep
        self.backbone = original_wrap.backbone
        
        # fix problem01
        # list of (stage, block) indices you already know are problematic
        TARGETS = [(0, 6), (1, 8), (2, 8), (3, 9), (4, 7)]
        for s_idx, b_idx in TARGETS:
            for tcm_idx in range(4):
                block = self.backbone.__getattr__(f"stage{s_idx}")[b_idx].tcm[tcm_idx]

                block.dwconvs[0] = Conv1dAs2d(block.dwconvs[0])
                block.pwconv1    = Conv1dAs2d(block.pwconv1)   # 1×1 conv

        
        # Replace ASTP with RKNN-safe version:
        self.pool = original_wrap.pool
        self.bn = original_wrap.bn
        self.linear = original_wrap.linear

    def forward(self, x):
        # x: shape [B, 1, n_mels, time_frames]
        # (1) Pass through the backbone
        x = self.backbone(x)    # shape might become [B, channels, frames] or similar
        # (2) Pooling
        x = self.pool(x)        # ASTP => shape likely [B, embedding_dim]
        # (3) BatchNorm
        x = self.bn(x)
        # (4) Final linear => 192-dim (if that's your embedding size)
        x = self.linear(x)
        return x


# Create an instance of our new model that skips the MelBanks front-end
model_no_mel = ReDimNetNoMel(original_model)



In [7]:
model_no_mel.eval()  # <- this line is critical!
dummy = torch.randn(1, 1, 60, 200)
model_no_mel(dummy)

tensor([[ 1.3411e+00,  1.1828e+00, -2.4811e+00, -2.2406e+00, -8.6126e-01,
          3.0935e-01,  1.2324e+00,  1.0377e+00,  1.6560e+00,  3.4042e+00,
          2.0860e+00, -1.8308e-01, -2.9661e+00,  4.2020e-01, -3.8264e+00,
          1.0532e+00, -2.6622e+00, -2.8416e-01,  1.4150e+00, -1.8020e+00,
         -4.9002e-02,  1.6581e+00, -7.3962e-01,  1.5025e+00,  3.3698e-01,
         -1.0475e+00,  1.3848e+00, -1.2775e+00, -1.0864e-01,  9.5220e-01,
          1.1822e+00, -3.5231e+00, -3.7416e-03,  1.5678e+00,  3.1225e-02,
          1.9735e+00,  1.8698e+00, -2.6484e+00, -1.1098e+00, -3.1876e+00,
         -2.1200e+00,  2.5363e+00,  3.3526e+00, -2.9072e+00,  3.5248e+00,
          1.4229e+00, -1.0935e+00, -3.3047e-01, -1.1186e+00,  1.4684e+00,
          5.7685e-01, -4.1560e-01, -2.5260e+00,  2.8701e+00, -3.8513e+00,
         -1.7056e+00, -1.8509e+00, -2.1664e+00,  2.0554e-01, -2.2015e+00,
         -1.8483e+00, -1.6807e+00,  3.9092e-01, -9.1267e-01,  7.6623e-01,
          2.7491e+00, -2.4884e+00,  2.

## layres debug

problematic layer tree is:

```
TimeContextBlock1d
├── red_dim_conv (Sequential)
│   ├── Conv1d(600 → 60, kernel_size=1)
│   └── LayerNorm(C=60, data_format=channels_first)
├── tcm (Sequential)
│   ├── ConvNeXtLikeBlock (kernel=7)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=7, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=19)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=19, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=31)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=31, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=59)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=59, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   └── TransformerEncoderLayer
│       ├── attention (MultiHeadAttention)
│       │   ├── k_proj: Linear(60 → 60)
│       │   ├── v_proj: Linear(60 → 60)
│       │   ├── q_proj: Linear(60 → 60)
│       │   └── out_proj: Linear(60 → 60)
│       ├── layer_norm: LayerNorm(60)
│       ├── feed_forward
│       │   ├── intermediate_dropout: Dropout(0.0)
│       │   ├── intermediate_dense: Linear(60 → 60)
│       │   ├── intermediate_act_fn: NewGELUActivation
│       │   ├── output_dense: Linear(60 → 60)
│       │   └── output_dropout: Dropout(0.0)
│       └── final_layer_norm: LayerNorm(60)
└── exp_dim_conv: Conv1d(60 → 600, kernel_size=1)
```

In [8]:
for name, module in model_no_mel.named_modules():
    if isinstance(module, nn.LayerNorm):
        print("❌ Still has LayerNorm at:", name)

❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.final_layer_norm


In [9]:
print("stage0.6 =", model_no_mel.backbone.stage4[7].tcm[0]) 

stage0.6 = ConvNeXtLikeBlock(
  (dwconvs): ModuleList(
    (0): Conv1dAs2d(
      (conv2d): Conv2d(60, 60, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0), groups=60)
    )
  )
  (norm): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): GELU(approximate='none')
  (pwconv1): Conv1dAs2d(
    (conv2d): Conv2d(60, 60, kernel_size=(1, 1), stride=(1, 1))
  )
)


## FP16 check


In [10]:
with torch.no_grad():
    fp16_net = copy.deepcopy(model_no_mel).half().eval()
    ok = torch.isfinite(fp16_net(dummy.half())).all()
    print('safe in pure FP16?', ok)

safe in pure FP16? tensor(True)


## info

In [11]:
model_no_mel.eval()


ReDimNetNoMel(
  (backbone): ReDimNet(
    (stem): Sequential(
      (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
      (2): to1d()
    )
    (stage0): Sequential(
      (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
      (1): to2d(f=60,c=10)
      (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
      (3): ConvBlock2d(
        (conv_block): ResBasicBlock(
          (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [12]:
summary(model_no_mel, (1, 1, 60, 200))


Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetNoMel                                                [1, 192]                  --
├─ReDimNet: 1-1                                              [1, 600, 200]             --
│    └─Sequential: 2-1                                       [1, 600, 200]             --
│    │    └─Conv2d: 3-1                                      [1, 10, 60, 200]          100
│    │    └─LayerNorm: 3-2                                   [1, 10, 60, 200]          20
│    │    └─to1d: 3-3                                        [1, 600, 200]             --
│    └─Sequential: 2-2                                       [1, 600, 200]             --
│    │    └─weigth1d: 3-4                                    [1, 600, 200]             (1)
│    │    └─to2d: 3-5                                        [1, 10, 60, 200]          --
│    │    └─Conv2d: 3-6                                      [1, 10, 60, 200]          110
│ 

## Utility Function for WAV -> MelSpectrogram

In [13]:

# ------------------------------------------------------------------
#  ReDimNet front-end settings (taken from the IDRnD repo defaults)
#    • 16 kHz audio
#    • pre-emphasis α = 0.97
#    • 25 ms window  (400 samples)
#    • 15 ms hop     (240 samples)  ➜ 134 frames for a 2-s clip
#    • 60 Mel bins, 20 Hz → 8 kHz
# ------------------------------------------------------------------
_PREEMPH  = 0.97
_SR       = 16_000
_N_FFT    = 512
_WIN_LEN  = 400
_HOP      = 240
_N_MELS   = 60
_F_MIN    = 20.0
_F_MAX    = 7600.0
_EPS      = 1e-6            # numerical stability


# Singleton MelSpectrogram so we build the kernel only once
_mel_layer = T.MelSpectrogram(
    sample_rate=_SR,
    n_fft=_N_FFT,
    win_length=_WIN_LEN,
    hop_length=_HOP,
    f_min=_F_MIN,
    f_max=_F_MAX,
    n_mels=_N_MELS,
    power=2.0,               # the original uses power-spec → log10 later
    center=True,
    pad_mode="reflect",
    window_fn=torch.hamming_window
)

def _pre_emphasis(wave: torch.Tensor, alpha: float = _PREEMPH) -> torch.Tensor:
    """y[n] = x[n] − α·x[n−1] (first sample unchanged)."""
    y = wave.clone()
    y[:, 1:] = y[:, 1:] - alpha * y[:, :-1]
    return y


def pad_or_crop_logmel(log_mel, target_frames=200):
    """
    Ensures log_mel is shaped [1, n_mels, target_frames] by:
    - Padding with zeros on the right if too short
    - Center-cropping if too long
    """
    B, M, T = log_mel.shape
    if T < target_frames:
        pad_amt = target_frames - T
        log_mel = F.pad(log_mel, (0, pad_amt))  # pad at end
        print(f"Padding log_mel from {T} to {target_frames} frames")
    elif T > target_frames:
        start = (T - target_frames) // 2
        log_mel = log_mel[:, :, start:start + target_frames]
        print(f"Cropping log_mel from {T} to {target_frames} frames")
    return log_mel


@torch.no_grad()
def waveform_to_logmel(wave: torch.Tensor) -> torch.Tensor:
    """
    Parameters
    ----------
    wave : Tensor [B', T] | [1, T]
        16-kHz mono waveform already trimmed / padded (32 000 samples for 2 s).

    Returns
    -------
    log_mel : Tensor [B', 1, 60, frames]
        Bit-exact front-end output expected by `model_no_mel`.
    """
    
    wave = wave[:,:32000]
    print(f"Input waveform shape: {wave.shape}")
    
    # Make sure we always have a batch dimension
    if wave.dim() == 1:      # (T,) → (1, T)
        wave = wave.unsqueeze(0)
    elif wave.dim() == 2 and wave.shape[0] > 1:
        raise ValueError("Input must be mono; got multi-channel tensor.")

    # pre-emphasis
    wave = _pre_emphasis(wave.float())

    # Mel power-spectrogram
    mel = _mel_layer(wave)
    mel = torch.log(mel + 1e-6)          # → [B, 60, frames]

    # log-scale (natural or log10 both work – log10 matches repo)
    mel = mel - mel.mean(dim=-1, keepdim=True)

    # pad/crop
    mel = pad_or_crop_logmel(mel, target_frames=134)  # Ensure 200 frames

    # add the dummy channel dim expected by Conv2d stem
    mel = mel.unsqueeze(1)                # → [B, 1, 60, frames]

    return mel

In [14]:
def example_inference(wav_path: str):
    # (a) Load audio
    waveform, sample_rate = torchaudio.load(wav_path)  # shape: [channels, time]
    # If stereo, select one channel, or average:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if needed
    target_sample_rate=16000
    if sample_rate != target_sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    # (b) Convert to log-mel
    log_mel = waveform_to_logmel(waveform)
    print('feeding logmel shape:', log_mel.shape)
    
    # (c) Forward pass
    with torch.no_grad():
        embedding = model_no_mel(log_mel)  # shape typically [1, 192] or so

    print("Embedding shape:", embedding.shape)
    #print("Embedding:", embedding)
    return embedding

In [15]:
# Compute similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    return F.cosine_similarity(embedding1, embedding2).item()

def cosine_similarity_numpys(emb1: np.ndarray, emb2: np.ndarray) -> float:
    """
    Compute cosine similarity between two vectors of shape (D,) or (1, D).
    """
    # If shape is (1, D), flatten to (D,)
    v1 = emb1.flatten()
    v2 = emb2.flatten()

    # dot product
    dot = np.dot(v1, v2)
    # norms
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)

    # Add a small epsilon in case of very small norms
    sim = dot / (norm1 * norm2 + 1e-8)
    return sim


In [16]:
embed0 = example_inference("test000.wav")
embed1 = example_inference("testRob1.wav")
embed2 = example_inference("testRob2.wav")
embed3 = example_inference("testme1.wav")
embed4 = example_inference("testme2.wav")


Input waveform shape: torch.Size([1, 32000])
feeding logmel shape: torch.Size([1, 1, 60, 134])
Embedding shape: torch.Size([1, 192])
Input waveform shape: torch.Size([1, 25776])
Padding log_mel from 108 to 134 frames
feeding logmel shape: torch.Size([1, 1, 60, 134])
Embedding shape: torch.Size([1, 192])
Input waveform shape: torch.Size([1, 23570])
Padding log_mel from 99 to 134 frames
feeding logmel shape: torch.Size([1, 1, 60, 134])
Embedding shape: torch.Size([1, 192])
Input waveform shape: torch.Size([1, 28126])
Padding log_mel from 118 to 134 frames
feeding logmel shape: torch.Size([1, 1, 60, 134])
Embedding shape: torch.Size([1, 192])
Input waveform shape: torch.Size([1, 32000])
feeding logmel shape: torch.Size([1, 1, 60, 134])
Embedding shape: torch.Size([1, 192])


In [17]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to webvoice): {cosine_similarity_numpys(embed0, embed1)}")
print(f"Similarity (robot to me1   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (robot to me2  ): {cosine_similarity_numpys(embed1, embed4)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")

Similarity (robot to robot): 0.8274897933006287
Similarity (robot to webvoice): 0.16069643199443817
Similarity (robot to me1   ): -0.021272331476211548
Similarity (robot to me2  ): 0.004241504706442356
Similarity (me 1 to me 2  ): 0.33580610156059265


/tmp/ipykernel_4074481/3060022475.py:14: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  dot = np.dot(v1, v2)


In [18]:
embed1[0][:10]

tensor([-0.5160,  2.1394, -0.5878, -3.4669, -5.8041, -3.1412,  1.8592,  0.8813,
        -1.9214, -3.0204])

## store

In [19]:
class NHWCWrapper(nn.Module):
    def __init__(self, model_nchw):
        super().__init__()
        self.model = model_nchw

    def forward(self, x):
        # x: NHWC => NCHW
        x = x.permute(0, 3, 1, 2).contiguous()
        return self.model(x)

In [20]:
import onnx

def export_to_onnx(model, onnx_path="ReDimNet_no_mel.onnx"):
    model.eval()
    
    # Create a dummy input with shape matching [B=1, 1, n_mels=60, time_frames=134] (example)
    dummy_input = torch.randn(1, 1, 60, 134)

    model_NHWC = NHWCWrapper(model)
    ## TODO: rand clip [8/-8]
    dummy_input_NHWC = torch.rand(1, 60, 134, 1)*16-8

    
    #  fixed-length segments 
    torch.onnx.export(
        model_NHWC,
        dummy_input_NHWC,
        onnx_path,
        input_names=["log_mel"],
        output_names=["embedding"],
        opset_version=17
    )
    
    # # dynamic axes for variable time frames
    # torch.onnx.export(
    #     model_NHWC,
    #     dummy_input_NHWC,
    #     onnx_path,
    #     input_names   = ["log_mel"],
    #     output_names  = ["embedding"],
    #     opset_version = 17,               # use a recent opset
    #     dynamic_axes = {
    #         # input  tensor : {axis_index : symbolic_name}
    #         "log_mel"  : {0: "batch",   2: "time"},   # B and T now flexible
    #         "embedding": {0: "batch"}                 # output length is fixed, batched
    #     }
    # ) 
    
    print("Exported to", onnx_path)
    



In [21]:
export_to_onnx(model_no_mel,onnx_path = "ReDimNet_no_mel.onnx")
!ls -lah ReDimNet_no_mel.onnx

Exported to ReDimNet_no_mel.onnx
-rw-rw-r-- 1 vlad vlad 4.2M Jun 17 14:04 ReDimNet_no_mel.onnx


### store half

In [22]:
model_NHWC = NHWCWrapper(model_no_mel)
dummy_input_NHWC = torch.rand(1, 60, 200, 1)*16-8

fp16_net = copy.deepcopy(model_NHWC).half().eval()
fp16_dummy = dummy_input_NHWC.half()

#  fixed-length segments 
torch.onnx.export(
   fp16_net,
   fp16_dummy,
   "ReDimNet_no_mel_half.onnx",
   input_names=["log_mel"],
   output_names=["embedding"],
   opset_version=13
)

PyTorch's .half() is unreliable for full model precision control in ONNX. Instead:

In [23]:
from onnxconverter_common.float16 import convert_float_to_float16
import onnx

model_fp32 = onnx.load("ReDimNet_no_mel.onnx")
model_fp16 = convert_float_to_float16(model_fp32, keep_io_types=True)
onnx.save(model_fp16, "ReDimNet_no_mel_fp16.onnx")


/data/proj/voice/pyvoice_venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 5.605193857299268e-45 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/data/proj/voice/pyvoice_venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 1.2503155177867598e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/data/proj/voice/pyvoice_venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 5.4816506287478153e-14 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/data/proj/voice/pyvoice_venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 2.2374582096875482e-17 will be truncated to 1e

## verify

In [24]:
# onnx_model_path = "ReDimNet_no_mel.onnx"
onnx_model_path = "ReDimNet_no_mel_fp16.onnx"

In [25]:
import onnx
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")


ONNX model is valid!


In [26]:
import onnxruntime as ort

def run_inference_onnx(onnx_path, wav_path):
    """
    Loads an audio file, converts to log-mel, and runs inference
    in an ONNX session. Returns the embedding as a NumPy array.
    """
    print("===================================================")
    print("===========   run_inference_onnx   ================")
    print("===================================================")
    #######################################
    # 1) Load your ONNX model
    #######################################
    # (Optional) onnx.checker to confirm it’s valid
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print(f"Loaded and checked ONNX model from: {onnx_path}")

    # Create an inference session
    session = ort.InferenceSession(onnx_path)

    # Usually we retrieve the first input & output name
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    #######################################
    # 2) Load audio, get log-mel
    #######################################
    print("loading audio from:", wav_path)
    waveform, sample_rate = torchaudio.load(wav_path)
    print(f"...Waveform rate {sample_rate}  ; shape : {waveform.shape}")

    
    # If multi-channel, downmix:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
        
    # Resample if needed
    target_sample_rate=16000
    if sample_rate != target_sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)
        # save resampled waveform to files with suffix "_resampled_16.wav"
        # torchaudio.save(wav_path.replace(".wav", "_resampled_16.wav"), waveform, target_sample_rate)

    log_mel = waveform_to_logmel(waveform)

    #######################################
    # 3) ONNX Inference
    #######################################
    # Convert to NumPy for ONNX runtime
    log_mel_np = log_mel.cpu().numpy()
    log_mel_np = np.transpose(log_mel_np, (0, 2, 3, 1))
    
    ## save log_mel_np to file with suffix "_logmel.npy" to check later
    print("logmelshape : ", log_mel_np.shape)
    np.save(wav_path.replace(".wav", "_logmel.npy"), log_mel_np)
    
    # Run inference
    outputs = session.run([output_name], {input_name: log_mel_np})
    # outputs is a list; typically we want the first item
    embedding = outputs[0]  # shape is [1, embedding_dim]

    # print("Embedding[10]: ", embedding[0:10])  # Print the 10th element of the embedding
    print("Embedding shape:", embedding.shape)
    # print("Embedding data:\n", embedding)
    return embedding


In [27]:

embed0 = run_inference_onnx(onnx_model_path, "test000.wav")
embed1 = run_inference_onnx(onnx_model_path, "testRob1.wav")
embed2 = run_inference_onnx(onnx_model_path, "testRob2.wav")
embed3 = run_inference_onnx(onnx_model_path, "testme1.wav")
embed4 = run_inference_onnx(onnx_model_path, "testme2.wav")
embed5 = run_inference_onnx(onnx_model_path, "pas_1.wav")
embed6 = run_inference_onnx(onnx_model_path, "pas_2.wav")


===========   run_inference_onnx   ================
Loaded and checked ONNX model from: ReDimNet_no_mel_fp16.onnx


2025-06-17 14:04:17.316474828 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'
2025-06-17 14:04:17.343660967 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'


loading audio from: test000.wav
...Waveform rate 16000  ; shape : torch.Size([1, 293699])
Input waveform shape: torch.Size([1, 32000])
logmelshape :  (1, 60, 134, 1)
Embedding shape: (1, 192)
===========   run_inference_onnx   ================
Loaded and checked ONNX model from: ReDimNet_no_mel_fp16.onnx


2025-06-17 14:04:18.130272970 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'
2025-06-17 14:04:18.153712581 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'


loading audio from: testRob1.wav
...Waveform rate 22050  ; shape : torch.Size([1, 35522])
Input waveform shape: torch.Size([1, 25776])
Padding log_mel from 108 to 134 frames
logmelshape :  (1, 60, 134, 1)
Embedding shape: (1, 192)
===========   run_inference_onnx   ================
Loaded and checked ONNX model from: ReDimNet_no_mel_fp16.onnx


...

Loaded and checked ONNX model from: ReDimNet_no_mel_fp16.onnx


2025-06-17 14:04:20.979767603 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'
2025-06-17 14:04:21.004148611 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'


loading audio from: pas_1.wav
...Waveform rate 16000  ; shape : torch.Size([1, 65867])
Input waveform shape: torch.Size([1, 32000])
logmelshape :  (1, 60, 134, 1)
Embedding shape: (1, 192)
===========   run_inference_onnx   ================
Loaded and checked ONNX model from: ReDimNet_no_mel_fp16.onnx


2025-06-17 14:04:22.008354035 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'
2025-06-17 14:04:22.033191727 [W:onnxruntime:, constant_folding.cc:268 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Sub node '/model/pool/Sub_1'


loading audio from: pas_2.wav
...Waveform rate 16000  ; shape : torch.Size([1, 101189])
Input waveform shape: torch.Size([1, 32000])
logmelshape :  (1, 60, 134, 1)
Embedding shape: (1, 192)


In [28]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to webvoice): {cosine_similarity_numpys(embed0, embed1)}")
print(f"Similarity (robot to me1   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (robot to me2  ): {cosine_similarity_numpys(embed1, embed4)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")
print(f"Similarity (pas 1 to pas 2  ): {cosine_similarity_numpys(embed5, embed6)}")

Similarity (robot to robot): 0.82741379737854
Similarity (robot to webvoice): 0.16026194393634796
Similarity (robot to me1   ): -0.021485380828380585
Similarity (robot to me2  ): 0.0041078138165175915
Similarity (me 1 to me 2  ): 0.33657512068748474
Similarity (pas 1 to pas 2  ): 0.5674994587898254


In [32]:
!md5sum pas_1_logmel.npy

a5d54ab150518b1c7709b4c69e662e0c  pas_1_logmel.npy


In [31]:
embed5[0][:10]

array([-1.7196702 , -2.4689784 ,  0.6681843 ,  0.74096453,  0.90874404,
        0.35022765, -1.50245   , -2.5797083 ,  0.72983295, -0.26111186],
      dtype=float32)

In [30]:
torch.save(embed5, '/tmp/refB0_embed5_for_our_redimnet.pt')


## cal fake data

* run in rknn docker:

``` bash

import os
import numpy as np
import torch

# Directory for calibration inputs
os.makedirs("calib_npy", exist_ok=True)

# Create 100 dummy log-mel tensors
for i in range(10):
    log_mel = torch.randn(1, 1, 60, 200).numpy().astype(np.float32)
    np.save(f"calib_npy/sample_{i}.npy", log_mel)

# Write dataset.txt listing all paths
with open("dataset.txt", "w") as f:
    for i in range(10):
        f.write(f"calib_npy/sample_{i}.npy\n")


````

## converts

* python convert.py ReDimNet_no_mel.onnx  rk3588 fp ReDimNet_no_mel.rknn 
* python convert.py ReDimNet_no_mel.onnx  rv1106 i8  ReDimNet_no_mel.rknn